In [ ]:
import random
import numpy as np
import tensorflow as tf

from utils.data_loader import get_train_test

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        tf.config.experimental.set_memory_growth(gpus[0], True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

In [ ]:
SEED = 0
IMG_DIM = 28

In [ ]:
random.seed(SEED)
tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
train_ds, val_ds = get_train_test(training=True)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer((IMG_DIM, IMG_DIM, 1)),
    tf.keras.layers.Conv2D(64, (3, 3), 1, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, (3, 3), 1, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, name='logits'),
    tf.keras.layers.Activation('softmax', name='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy'],
)

model.summary()

In [ ]:
model.fit(
    train_ds,
    epochs=5,
    validation_data=val_ds,
)

In [ ]:
model.save('models/mnist.h5')